# Exploring Convolutional Layers Through Data and Experiments


## SetUp

In [1]:
%pip install numpy matplotlib tensorflow

  Using cached numpy-2.4.2-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Using cached matplotlib-3.10.8-cp312-cp312-win_amd64.whl.metadata (52 kB)
  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp312-cp312-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-12.1.0-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.3.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  U

## Dataset Definition

In [ ]:
from tensorflow.keras.datasets import cifar100

# Cargar dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

print(x_train.shape)  # (50000, 32, 32, 3)
print(y_train.shape)  # (50000, 1)

CIFAR-100 is suitable for convolutional layers because it contains fixed-size RGB images (32×32×3), allowing for the exploitation of spatial locality and 2D structure. Multiple classes (100 categories) require learning hierarchical patterns (edges, textures, parts, and objects), something convolutions excel at thanks to the use of shared filters and translation-invariant feature learning. Furthermore, the size is small enough to efficiently experiment with and train CNN models while maintaining sufficient visual complexity to assess their capabilities.

## EDA 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Dataset Size and Class Distribution
print("="*60)
print("DATASET SIZE")
print("="*60)
print(f"Training samples: {x_train.shape[0]:,}")
print(f"Test samples: {x_test.shape[0]:,}")
print(f"Total samples: {x_train.shape[0] + x_test.shape[0]:,}")
print(f"\nNumber of classes: {len(np.unique(y_train))}")

print("CLASS DISTRIBUTION")

# Count samples per class
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

print(f"Samples per class (train): min={counts_train.min()}, max={counts_train.max()}, mean={counts_train.mean():.0f}")
print(f"Samples per class (test): min={counts_test.min()}, max={counts_test.max()}, mean={counts_test.mean():.0f}")

# Check if balanced
if counts_train.min() == counts_train.max():
    print("✓ Training set is perfectly balanced")
else:
    print("✗ Training set has class imbalance")

if counts_test.min() == counts_test.max():
    print("✓ Test set is perfectly balanced")
else:
    print("✗ Test set has class imbalance")

In [ ]:
# 2. Image Dimensions and Channels
print("\n" + "="*60)
print("IMAGE PROPERTIES")
print("="*60)
print(f"Image shape: {x_train.shape[1:]} (Height × Width × Channels)")
print(f"Height: {x_train.shape[1]} pixels")
print(f"Width: {x_train.shape[2]} pixels")
print(f"Channels: {x_train.shape[3]} (RGB)")
print(f"Data type: {x_train.dtype}")
print(f"Pixel value range: [{x_train.min()}, {x_train.max()}]")

# Calculate statistics per channel
print("\nPer-channel statistics (training set):")
for i, channel in enumerate(['Red', 'Green', 'Blue']):
    channel_data = x_train[:, :, :, i]
    print(f"  {channel}: mean={channel_data.mean():.2f}, std={channel_data.std():.2f}")

In [ ]:
# 3. Visualize Sample Images per Class
# CIFAR-100 class names
cifar100_classes = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

print("\n" + "="*60)
print("SAMPLE IMAGES")
print("="*60)

# Display samples from 10 random classes
np.random.seed(42)
sample_classes = np.random.choice(100, 10, replace=False)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for idx, class_id in enumerate(sample_classes):
    # Find first image of this class
    class_indices = np.where(y_train == class_id)[0]
    first_image_idx = class_indices[0]
    
    # Display image
    axes[idx].imshow(x_train[first_image_idx])
    axes[idx].set_title(f"Class {class_id}: {cifar100_classes[class_id]}", fontsize=9)
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle("Sample Images from 10 Random Classes", y=1.02, fontsize=14)
plt.show()

print(f"\nShowing 1 sample from 10 random classes (out of 100 total)")

In [ ]:
# 4. Within-Class Variation
print("\n" + "="*60)
print("WITHIN-CLASS VARIATION")
print("="*60)

# Show 5 samples from 3 different classes
selected_classes = [0, 50, 99]  # apple, mouse, worm
fig, axes = plt.subplots(3, 5, figsize=(12, 7))

for row, class_id in enumerate(selected_classes):
    class_indices = np.where(y_train == class_id)[0][:5]
    
    for col, img_idx in enumerate(class_indices):
        axes[row, col].imshow(x_train[img_idx])
        if col == 0:
            axes[row, col].set_ylabel(f"{cifar100_classes[class_id]}", fontsize=10, fontweight='bold')
        axes[row, col].axis('off')

plt.suptitle("5 Different Samples from Each Class (showing within-class variation)", fontsize=12)
plt.tight_layout()
plt.show()

print("Note: Variation within classes demonstrates the challenge of CIFAR-100")

In [ ]:
# 5. Preprocessing Recommendations
print("\n" + "="*60)
print("PREPROCESSING RECOMMENDATIONS")
print("="*60)

print("\n1. NORMALIZATION (Required)")
print("   Current range: [0, 255]")
print("   Recommended: Scale to [0, 1] by dividing by 255")
print("   Reason: Improves gradient descent convergence")

print("\n2. RESIZING")
print("   Current size: 32×32")
print("   Recommendation: NOT NEEDED - size is appropriate for CNNs")
print("   Note: Images are already small; upscaling may introduce artifacts")

print("\n3. DATA AUGMENTATION (Recommended for training)")
print("   Suggested augmentations:")
print("   • Horizontal flips (for objects that look similar when flipped)")
print("   • Random crops with padding")
print("   • Random brightness/contrast adjustments")
print("   • Cutout/Random erasing")
print("   Reason: Improves model generalization with limited data")

print("\n4. STANDARDIZATION (Optional)")
print("   Calculate per-channel mean and std across dataset")
print("   Apply: (x - mean) / std")
print("   Reason: Centers data around 0, can improve training stability")

print("\n5. LABEL ENCODING")
print("   Current shape:", y_train.shape, "→ needs one-hot encoding for multi-class")
print("   Or use sparse categorical crossentropy with integer labels")

print("\n" + "="*60)
print("EDA SUMMARY")
print("="*60)
print(f"✓ Dataset: CIFAR-100")
print(f"✓ Size: 50,000 train + 10,000 test = 60,000 total")
print(f"✓ Classes: 100 (balanced)")
print(f"✓ Images: 32×32×3 RGB")
print(f"✓ Pixel range: [0, 255] → needs normalization")
print(f"✓ Ready for CNN training with preprocessing")